<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#데이터-로드" data-toc-modified-id="데이터-로드-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>데이터 로드</a></span></li><li><span><a href="#인코더,-디코더-정의" data-toc-modified-id="인코더,-디코더-정의-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>인코더, 디코더 정의</a></span></li><li><span><a href="#임베딩-+-인코딩-+-디코딩을-결합하는-Transformer-Class-정의" data-toc-modified-id="임베딩-+-인코딩-+-디코딩을-결합하는-Transformer-Class-정의-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>임베딩 + 인코딩 + 디코딩을 결합하는 Transformer Class 정의</a></span></li><li><span><a href="#옵티마이저-설정하기" data-toc-modified-id="옵티마이저-설정하기-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>옵티마이저 설정하기</a></span></li><li><span><a href="#Loss-함수-설정하기" data-toc-modified-id="Loss-함수-설정하기-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Loss 함수 설정하기</a></span></li><li><span><a href="#모델-훈련" data-toc-modified-id="모델-훈련-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>모델 훈련</a></span></li><li><span><a href="#Inference" data-toc-modified-id="Inference-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Inference</a></span></li></ul></div>

# 데이터 로드
데이터 로드는 tensorflow 2에서 제공하는 tf.data.dataset 형식으로 로드하도록 하겠습니다. dataset 형식으로 데이터를 불러오면, 향후 모델을 훈련시킬 때 @tf.function 데코레이터를 활용하여 상당히 빠르게 학습할 수 있습니다.

In [3]:
!pip install -q tf-nightly

^C


In [1]:
n_layers = 4
d_model = 128
dff = 512
n_heads = 8
SEQ_LEN = 40
dk = d_model//n_heads

In [1]:
# import tensorflow_datasets as tfds
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import tensorflow as tf
from tensorflow.keras import Input, Model, Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Layer, Dense, Flatten, Dropout, Concatenate, Add, Dot, Multiply, Reshape, Activation, BatchNormalization, LayerNormalization, SimpleRNNCell, RNN, SimpleRNN, LSTM, Embedding, Bidirectional, TimeDistributed, Conv1D, Conv2D, MaxPool1D, MaxPool2D, GlobalMaxPool1D, GlobalMaxPool2D, AveragePooling1D, AveragePooling2D, GlobalAveragePooling1D, GlobalAveragePooling2D, ZeroPadding2D, RepeatVector
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras.optimizers import SGD, Adagrad, Adam
from tensorflow.keras.losses import MeanSquaredError, MeanAbsoluteError, MeanAbsolutePercentageError, BinaryCrossentropy, CategoricalCrossentropy, SparseCategoricalCrossentropy, CosineSimilarity
from tensorflow.keras.metrics import MeanSquaredError, RootMeanSquaredError, MeanAbsoluteError, MeanAbsolutePercentageError, BinaryCrossentropy, CategoricalCrossentropy, SparseCategoricalCrossentropy, CosineSimilarity
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.activations import linear, sigmoid, relu
from tensorflow.keras.initializers import RandomNormal, glorot_uniform, he_uniform, Constant

plt.style.use("dark_background")

ImportError: cannot import name '_pywrap_tf2' from 'tensorflow.python.platform' (C:\Users\00006363\Anaconda3\envs\tf2.3\lib\site-packages\tensorflow\python\platform\__init__.py)

In [4]:
examples, metadata = tfds.load("ted_hrlr_translate/pt_to_en", with_info=True, as_supervised=True)

Shuffling and writing examples to C:\/Users/00006363/tensorflow_datasets/ted_hrlr_translate/pt_to_en/1.0.0.incompleteNBZRR4/ted_hrlr_translate-train.tfrecord


Shuffling and writing examples to C:\/Users/00006363/tensorflow_datasets/ted_hrlr_translate/pt_to_en/1.0.0.incompleteNBZRR4/ted_hrlr_translate-validation.tfrecord


Shuffling and writing examples to C:\/Users/00006363/tensorflow_datasets/ted_hrlr_translate/pt_to_en/1.0.0.incompleteNBZRR4/ted_hrlr_translate-test.tfrecord


Dataset ted_hrlr_translate downloaded and prepared to C:\Users\00006363\tensorflow_datasets\ted_hrlr_translate\pt_to_en\1.0.0. Subsequent calls will reuse this data.


In [4]:
# 데이터가 test, train, validation에 있음
# 이 데이터들의 형식은 tf.data.dataset 오브젝트
examples

type(examples['train'])

# train, validation, test 데이터로 나누기
train_examples, val_examples, test_examples = examples["train"], examples["validation"], examples["test"]

{Split('train'): <PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.string, name=None))>,
 Split('validation'): <PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.string, name=None))>,
 Split('test'): <PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.string, name=None))>}

In [ ]:
# train_example 에는 포르투갈어 pt, 영어 en이 있음
# 여기서 단어를 추출해서, 단어들마다 인덱싱을 부여할 것임
# 인덱싱을 부여하는 클래스가 tokenizer_en(영어 인덱싱 부여), tokenizer_pt(포르투갈어 인덱싱 부여)
tokenizer_en = tfds.features.text.SubwordTextEncoder.build_from_corpus(
    (en.numpy() for pt, en in train_examples), target_vocab_size=2**13)
tokenizer_pt = tfds.features.text.SubwordTextEncoder.build_from_corpus(
    (pt.numpy() for pt, en in train_examples), target_vocab_size=2**13)

In [ ]:
# 영어를 인코딩
tokenizer_en.encode("Hello man, Let's run transformer!")

[7903, 2429, 439, 406, 7345, 7907, 1283, 7870, 9, 527, 6514, 7945, 7864]

In [ ]:
# 포르투갈어를 인코딩
print(tokenizer_pt.encode("vou então muito rapidamente partilhar convosco algumas histórias de algumas coisas mágicas que aconteceram."))

[141, 77, 33, 1566, 873, 4501, 217, 642, 4, 217, 101, 1073, 4824, 17, 5, 488, 200, 8004]


In [ ]:
def encode(lang1, lang2):
    lang1 = [tokenizer_pt.vocab_size] + tokenizer_pt.encode(
      lang1.numpy()) + [tokenizer_pt.vocab_size+1] # 포르투갈 어를 인코딩 할 때 시작 단어를 의미하는 숫자와, 끝 단어를 의미하는 숫자가 붙음

    lang2 = [tokenizer_en.vocab_size] + tokenizer_en.encode(
      lang2.numpy()) + [tokenizer_en.vocab_size+1] # 영어도 마찬가지임
    return lang1, lang2

In [ ]:
lang1 = tf.constant("vou então muito rapidamente partilhar convosco algumas histórias de algumas coisas mágicas que aconteceram.", dtype=tf.string)
lang2 = tf.constant("Hello man, Let's run transformer!", dtype=tf.string)

In [ ]:
print(encode(lang1, lang2))

([8214, 141, 77, 33, 1566, 873, 4501, 217, 642, 4, 217, 101, 1073, 4824, 17, 5, 488, 200, 8004, 8215], [8087, 7903, 2429, 439, 406, 7345, 7907, 1283, 7870, 9, 527, 6514, 7945, 7864, 8088])


In [ ]:
# tf.py_function을 활용하여 모든 문장에 숫자를 부여함

def tf_encode(pt, en):
    result_pt, result_en = tf.py_function(encode, [pt, en], [tf.int64, tf.int64]) # 인풋 값으로 pt와 en이 한문장 한문장씩 들어가서 result_pt, result_en을 아웃풋으로 내게 됨
    result_pt.set_shape([None])
    result_en.set_shape([None])
    return result_pt, result_en

In [ ]:
MAX_LENGTH = 40
BUFFER_SIZE = 60000
BATCH_SIZE = 64

# 전체 데이터에서 x가 포르투갈어 한 문장, y가 영어 한 문장인데
# 만약 문장이 인코딩 되었을 때, 인코딩 된 길이가 40을 초과하면 데이터에서 배제하고자 하는 필터 함수임
def filter_max_length(x, y, max_length=MAX_LENGTH):
    return tf.logical_and(tf.size(x) <= max_length,
                        tf.size(y) <= max_length)

In [ ]:
train_dataset = train_examples.map(tf_encode) # tf_encode 함수를 활용해서 포르투갈어, 영어 각 문장에 시작 토큰과 끝 토큰을 부여함
train_dataset = train_dataset.filter(filter_max_length) # 문장의 길이가 40이 넘는 문장은 배제하고자 함
# cache the dataset to memory to get a speedup while reading from it.
train_dataset = train_dataset.cache() # cache를 활용해서 데이터를 로드할 때 빠른 처리를 기대해 봄
train_dataset = train_dataset.shuffle(BUFFER_SIZE).padded_batch(BATCH_SIZE) 
# shuffle(60000) 인데, 이 말은 전체 데이터를 완전히 섞겠다는 뜻임
# 전체 데이터의 수가 50000인데 50000보다 큰 숫자를 입력하면 완전하게 전체 데이터를 섞는 것이며
# 전체 데이터 수보다 작은 수를 입력하면 전체 데이터에서 일부만 섞음
# padded_batch는 무엇이냐면, 이번 데이터셋은 문장마다 길이가 모두 다르기에
# 배치 사이즈(32) 만큼의 문장을 뽑을 때마다
# 배치 사이즈에 해당하는 만큼의 문장의 길이는 일정하게 유지됨
# 무슨 말이냐면 배치가 2개라면  이 중 하나의 문장의 길이는 37이 될 수 있고
# 두 개의 배치(32) 중 하나의 배치는 문장의 길이를 37개로 모두 유지
# 그 다음 배치에서 문장의 길이가 39 라면, 그 배치에서는 문장의 길이를 39로 유지
                                                                            
train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE) # 데이터 로드와 처리의 시간을 overlap하여 속도 향상

val_dataset = val_examples.map(tf_encode)
val_dataset = val_dataset.filter(filter_max_length).padded_batch(BATCH_SIZE) #가변길이의 배치를 돌릴때 꼭 쓰자

- ![Imgur](https://i.imgur.com/Tl2zsFL.png)
- ![Imgur](https://i.imgur.com/SNIEhlA.png)

In [33]:
d_model = 512
def positional_encoding_matrix(seq_len, d_model):
    a, b = np.meshgrid(np.arange(d_model), np.arange(seq_len))
    pe_mat = b/10000**(2*(a//2)/d_model)
    pe_mat[:, 0::2] = np.sin(pe_mat[:, 0::2])
    pe_mat[:, 1::2] = np.cos(pe_mat[:, 1::2])
    pe_mat = pe_mat[None, :]
    return pe_mat

class TransformerEmbedding(Layer):
    def __init__(self, input_vocab_size, max_pe_dim):
        super().__init__()
        self.embedding = Embedding(input_dim=input_vocab_size, output_dim=d_model)
         # vocab_size는 tokenizer 내부 vocab.txt의 사이즈
        self.pe_mat = positional_encoding_matrix(max_pe_dim, d_model)
    
    # `x`: (batch_size, seq_len)
    def call(self, x, training):
        seq_len = tf.shape(x)[1]
        # (batch_size, seq_len, d_model)
        z = self.embedding(x)
        # 포지셔널 인코딩은 순서만을 의미하기 때문에 그 영향을 줄이도록 합니다.
        z = (d_model**0.5)*z + self.pe_mat[:, :seq_len, :]
        z = Dropout(rate=0.1)(z, training=training)
        # (batch_size, seq_len, d_model)
        return z

# d_model = 512
# x = np.float32(np.random.uniform(size=(1, 40))) # 문장 길이 40
# embedded = TransformerEmbedding(100 + 2, 10000)(x, False)
# print(embedded) # 문장이 위치 임베딩 + 포지션 임베딩

In [102]:
def scaled_dot_product_attention(queries, keys, values, mask=None):
    attn_scores = tf.matmul(queries, keys, transpose_b=True)/dk**0.5
    if mask is not None:
        attn_scores = attn_scores + (mask*-1e9)
    # (batch_size, seq_len_dec, seq_len_enc)
    attn_weights = tf.nn.softmax(attn_scores, axis=-1)
    # (batch_size, seq_len_dec, dk) (Same shape as queries)
    context_vec = tf.matmul(attn_weights, values)
    return context_vec, attn_weights

class MultiheadAttention(Layer):
    def __init__(self):
        super().__init__()
#         self.dense = Dense(units=d_model)

    def split_heads(self, x):
        x = tf.reshape(x, shape=(batch_size, -1, n_heads, dk))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, values, keys, queries, mask):
        queries = Dense(units=d_model)(queries)
        keys = Dense(units=d_model)(keys)
        values = Dense(units=d_model)(values)

        batch_size = tf.shape(queries)[0]
        # (batch_size, n_heads, seq_len_dec, dk)
        queries = self.split_heads(queries)
        # (batch_size, n_heads, seq_len_enc, dk)
        keys = self.split_heads(keys)
        # (batch_size, n_heads, seq_len_enc, dk)
        values = self.split_heads(values)

        # (batch_size, n_heads, seq_len_dec, dk)
        context_vec, attn_weights = scaled_dot_product_attention(queries, keys, values, mask)
        # (batch_size, seq_len_dec, n_heads, dk)
        z = tf.transpose(context_vec, perm=[0, 2, 1, 3])
        # (batch_size, seq_len_dec, d_model)
        z = tf.reshape(z, shape=(batch_size, -1, d_model))
        z = Dense(units=d_model)(z)
        return z, attn_weights

In [109]:
# q = tf.cast(np.random.uniform(size=(1, 2, d_model)), dtype=tf.float32)
# k = tf.cast(np.random.uniform(size=(1, 7, d_model)), dtype=tf.float32)
# v = tf.cast(np.random.uniform(size=(1, 7, d_model)), dtype=tf.float32)
# encoder_output, _ = MultiheadAttention()(v, k, q, mask=None)
# print(encoder_output.shape)
# # q = split_heads(q)
# # k = split_heads(k)
# # v = split_heads(v)

# # print(scaled_dot_product_attention(q, k, v, None)[0].shape)
# # print(scaled_dot_product_attention(q, k, v, None)[1].shape)

(1, 2, 128)


# 인코더, 디코더 정의
지금까지 임베딩 -> 멀티 헤드 어텐션 -> 포인트와이즈 피드 포워드 네트워크까지 살펴 보았습니다.  
트랜스포머에서의 인코딩은 (멀티헤드어텐션 + 포인트와이즈 피드 포워드 네트워크)를 층층이 쌓은 것입니다. 따라서 방금 공부했던 멀티헤드 어텐션과 포인트와이즈 피드 포워드 네트워크를 겹겹이 쌓아서, 인코더층을 쌓는 것입니다.  

인코더 레이어를 정의해서 멀티헤드 어텐션과 포인트와이즈 피드 포워드 네트워크를 합쳐 보도록 하겠습니다. 

단어 임베딩 벡터(단어 임베딩+위치 임베딩)가 -> 1. 멀티헤드어텐션  -> 2. Residual Network를 거쳐 원래 input과 멀티헤드어텐션의 합이 출력 -> 3. 포인트와이즈 피드포워드 네트워크 를 거치고, -> 4. Residual Network를 거쳐서 원래의 인풋과 포인트와이즈 피드포워드 네트워크의 합이 출력됨  
![Imgur](https://i.imgur.com/w4n19Rs.png)

디코더를 정의하도록 하겠습니다.  
![Imgur](https://i.imgur.com/kGTfPr8.png)

디코더는 인코더랑 유사하지만, 구조가 약간 다릅니다. 
이번 Seq2Seq는 포르투갈어를 영어로 바꾸는 문제입니다.  디코더에서는 
두단계의 멀티 헤드 어텐션 구조를 거치는데, **첫번째 멀티 헤드 어텐션은**, **영어문장과 영어문장의 셀프 어텐션을 하여**, 영어 문장간의 관계를 배우게 됩니다.  
두번째 멀티 헤드 어텐션은 **포르투갈어가 인코딩 된 것과**, **영어 문장간의 셀프** **어텐션된 결과를 다시 어텐션 해서 포르투갈 어와 영어의 관계를 학습하게 됩니다.**  

포르투갈어가 암호화된 것과, 영어 문장 한단어 한단어를 보면서 다음 단어를 예측하게 되기 때문에, look_ahead_mask를 사용하게 됩니다.  
만약 영어 문장이 (I love you) 로 이루어져 있다면, look_ahead_mask를 사용하면,  
(I, 0, 0) -> Love 예측, (I love, 0) -> You 예측, (I love you) -> 단어의 끝인 [SEP] 예측을 합니다.   
**즉 look_ahead_mask는 다음 단어를 예측할 때, 전에 있던 단어만으로 예측할수 있도록 앞에 있는 단어는 가리는 것입니다.**
  
  이러한 역할을 가능하게 하는 mask가 look_ahead_mask 입니다.

look_ahead_mask를 알아보겠습니다.**(매우 중요)**  
참고로 패딩은 1로 하겠습니다. 왜냐하면 어텐션 부분에서 mask * (-1e9)를 하는데, 패딩이 1이어야 -1e9가 곱해져서 상당히 음수로 큰 수가 되는 것이고, 이게 소프트 맥스에 들어가면 0이 되기 때문입니다.(지수함수라 지수함수에 -음수는 0으로 수렴)  
![Imgur](https://i.imgur.com/eLAlzji.png)  
![Imgur](https://i.imgur.com/gAVenk0.png)  
![Imgur](https://i.imgur.com/hsZ6dGs.png)

In [ ]:
# 상삼각행렬 만들기
tf.linalg.band_part(tf.ones((10, 10)), -1, 0)

<tf.Tensor: shape=(10, 10), dtype=float32, numpy=
array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
       [1., 1., 1., 1., 0., 0., 0., 0., 0., 0.],
       [1., 1., 1., 1., 1., 0., 0., 0., 0., 0.],
       [1., 1., 1., 1., 1., 1., 0., 0., 0., 0.],
       [1., 1., 1., 1., 1., 1., 1., 0., 0., 0.],
       [1., 1., 1., 1., 1., 1., 1., 1., 0., 0.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 0.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]], dtype=float32)>

In [ ]:
temp_mask = 1 - tf.linalg.band_part(tf.ones((40, 40)), -1, 0)
print(temp_mask)

tf.Tensor(
[[0. 1. 1. ... 1. 1. 1.]
 [0. 0. 1. ... 1. 1. 1.]
 [0. 0. 0. ... 1. 1. 1.]
 ...
 [0. 0. 0. ... 0. 1. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 0.]], shape=(40, 40), dtype=float32)


In [ ]:
# 예제 문장 (1, 40) 즉, 1문장, 40개의 단어를 가짐
example_sentence = np.hstack([np.random.randint(20, size=10), np.zeros(30)])[np.newaxis, :]
print(example_sentence) # 예제 문장
example_sentence = tf.cast(tf.math.equal(example_sentence, 0), dtype=tf.float32)
print(example_sentence) # 패딩 된 것(문장에서 0이 아닌 부분은 0으로, 0인 부분은 1로)
example_sentence = example_sentence[:, tf.newaxis, tf.newaxis, :] # 차원 변경
print(example_sentence)

look_ahead_mask = tf.maximum(temp_mask, example_sentence) # 상삼각행렬과 example sentence를 비교해가며 최대값만 취해서 패딩을 1로 처리함
# look ahead mask
print(look_ahead_mask)

[[15.  9.  5. 15. 16. 14. 15.  8. 13. 12.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.]]
tf.Tensor(
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]], shape=(1, 40), dtype=float32)
tf.Tensor(
[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
    1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]]]], shape=(1, 1, 1, 40), dtype=float32)
tf.Tensor(
[[[[0. 1. 1. ... 1. 1. 1.]
   [0. 0. 1. ... 1. 1. 1.]
   [0. 0. 0. ... 1. 1. 1.]
   ...
   [0. 0. 0. ... 1. 1. 1.]
   [0. 0. 0. ... 1. 1. 1.]
   [0. 0. 0. ... 1. 1. 1.]]]], shape=(1, 1, 40, 40), dtype=float32)


In [ ]:
# look_ahead_mask 알아보기
def create_look_ahead_mask(size):
  mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
  return mask
  
def create_masks(tar):
  temp_mask = create_look_ahead_mask(tf.shape(tar)[1])
  
  
  reverse_tar = tf.cast(tf.math.equal(tar, 0), dtype=tf.float32)
  reverse_tar = reverse_tar[:,tf.newaxis,tf.newaxis,:]
  look_ahead_mask = tf.maximum(reverse_tar, temp_mask)


  return look_ahead_mask

또한 패딩 마스크를 두어서, 패딩인 부분은 1으로 처리하겠습니다.

In [ ]:
def create_padding_mask(seq):
  seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
  return seq[:, tf.newaxis, tf.newaxis, :]  # (batch_size, 1, 1, seq_len)

In [ ]:
example_sentence = np.hstack([np.random.randint(20, size=10), np.zeros(30)])[np.newaxis, :]
# 패딩 되기 전
print(example_sentence)
# 패딩 된 후
print(create_padding_mask(example_sentence))

[[10.  1.  4. 16.  3.  9. 18.  6. 17. 13.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.]]
tf.Tensor(
[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
    1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]]]], shape=(1, 1, 1, 40), dtype=float32)


In [ ]:
class DecoderLayer(Layer):
  def __init__(self, d_model, n_heads, dff, rate=0.1):
    super().__init__()

    self.mha1 = MultiheadAttention(d_model, n_heads)
    self.mha2 = MultiheadAttention(d_model, n_heads)
    
    self.ffn = PositionwiseFFNN(d_model, dff)

    self.layernorm1 = LayerNormalization(epsilon=1e-6)
    self.layernorm2 = LayerNormalization(epsilon=1e-6)
    self.layernorm3 = LayerNormalization(epsilon=1e-6)

    self.dropout1 = Dropout(rate)
    self.dropout2 = Dropout(rate)
    self.dropout3 = Dropout(rate)


  def call(self, x, enc_output, training, padding_mask, look_ahead_mask):
    # x : 훈련 과정에서는 Seq2Seq에서 번역이 될 문장이 입력됨,
    # x : 추론 과정에서는 과거의 단어가 입력됨
    # enc_output : 인코더의 출력
    # padding_mask : 멀티 헤드 어텐션에 필요한 정보만 남기고 나머지는 패딩 처리
    # look_ahead_mask : 위에 설명
    # enc_output_shape == (batch_size, input_seq_len, d_model)

    #### 첫번째 멀티 헤드 어텐션 파트임 ####
    #### 첫번째 멀티 헤드 어텐션은, 인코더와의 결합 없이 번역이 될 문장끼리만 어텐션을 함 ####
    attn1, attn_weights_block1 = self.mha1(x, x, x, look_ahead_mask)
    out1 = self.dropout1(attn1, training=training)
    out1 = self.layernorm1(out1 + x)
    
    #### 두번째 멀티 헤드 어텐션 파트임 ####
    #### 두번째 멀티 헤드 어텐션 파트는 enc_output에서 인코더와(포르투갈어), out1(영어문장으로만 셀프어텐션을 한 것)
    #### 이 다시 멀티 헤드 어텐션 과정을 거치게 됨
    #### 다시 상기하자면, 이번 과제는 포르투갈 어를 영어로 번역하는 것임
    attn2, attn_weights_block2 = self.mha2(
        enc_output, enc_output, out1, padding_mask)
    out2 = self.dropout2(attn2, training=training)
    out2 = self.layernorm2(out2 + out1) # (batch_size, target_seq_len, d_model)
    

    ffn_output = self.ffn(out2)
    ffn_output = self.dropout3(ffn_output, training=training)
    out3 = self.layernorm3(ffn_output + out2) # (batch_size, target_seq_len, d_model)
    

    return out3, attn_weights_block1, attn_weights_block2

In [ ]:
sample_decoder_layer = DecoderLayer(d_model=512, n_heads=8, dff=2048)
sample_decoder_layer_output, _, _ = sample_decoder_layer(embedded, sample_encoding, False, None, None)
print(sample_decoder_layer_output)
print(sample_decoder_layer_output.shape)

tf.Tensor(
[[[-1.861443    0.27191493  2.616368   ... -0.06494223  0.37421167
   -0.33387262]
  [-1.4556838   0.02654802  2.9826684  ... -0.11393957  0.32178423
   -0.3594063 ]
  [-1.4244808  -0.52545476  2.9944925  ... -0.17711166  0.2963217
   -0.42363918]
  ...
  [-1.869552    0.39455086  2.1463938  ...  0.01799125  0.5466902
   -0.32442454]
  [-1.417133    0.5532684   2.2169363  ... -0.02880318  0.5430099
   -0.24101742]
  [-1.0640508   0.19279505  2.6072536  ... -0.03333719  0.5305211
   -0.1643059 ]]], shape=(1, 40, 512), dtype=float32)
(1, 40, 512)


In [1]:
class PositionwiseFFNN(Layer):
    def __init__(self):
        super().__init__()
        self.middle = Dense(dff, activation="relu")
        self.out = Dense(d_model)

    def call(self, x):
        # (batch_size, seq_len, dff)
        z = self.middle(x)
        # (batch_size, seq_len, d_model)
        z = self.out(z)
        return z

class EncoderLayer(Layer):
    def __init__(self, d_model, n_heads, dff, rate=0.1):
        super().__init__()
        self.mha = MultiheadAttention(d_model, n_heads)
        self.ffn = PositionwiseFFNN(d_model, dff)

        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)

    def call(self, x, training, mask=None):
        # x : 위치 임베딩 + 단어 임베딩 된 인코딩의 인풋
        z, _ = self.mha(x, x, x, mask) 
        # (batch_size, seq_len_enc, d_model)
        z = self.dropout1(z, training=training)
        # "Add & Normalize" Part
        out1 = self.layernorm1(x + z)

        # (batch_size, seq_len_enc, d_model)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        # "Add & Normalize" Part
        # (batch_size, seq_len_enc, d_model)
        out2 = self.layernorm2(out1 + ffn_output)
        return out2 

# 임베딩
x = np.float32(np.random.uniform(size=(1, 40))) # 문장 길이 40
Embedder = TransformerEmbedding(512, tokenizer_en.vocab_size+2, 10000)
embedded = Embedder(x, False)
print(embedded) # 문장이 위치 임베딩 + 포지션 임베딩
#인코더
sample_encoder = EncoderLayer(d_model=512, n_heads=8, dff=2048, rate=0.1)
sample_encoding = sample_encoder(embedded, training=None, mask=None)
print(sample_encoding)
#최종출력
print("Encoded 차원 :", sample_encoding.shape)

NameError: name 'Layer' is not defined

In [ ]:
class Encoder(Layer):
    def __init__(self, n_layers, d_model, n_heads, dff, rate=0.1):
        super().__init__()

        self.d_model = d_model
        self.n_layers = n_layers

        self.enc_layers = [EncoderLayer(d_model, n_heads, dff, rate) for _ in range(n_layers)] # 인코더를 쌓아서 층을 만듦

    def call(self, x, training, mask):
        # 인풋은 타겟 임베딩 + 포지셔닝 임베딩
        for i in range(self.n_layers):
            x = self.enc_layers[i](x, training, mask)
        # (batch_size, input_seq_len, d_model)
        return x

Encoder_layer = Encoder(6, 512, 8, 2048) #6층, 512차원의 단어 임베딩, 8개의 병렬 멀티 헤드 어텐션의 인코더
# embedded : 위치 임베딩 + 단어 임베딩
Encoded = Encoder_layer(embedded, False, None)
print(Encoded, Encoded.shape)

In [ ]:
class Decoder(Layer):
  def __init__(self, n_layers, d_model, n_heads, dff, rate=0.1):
    super().__init__()

    self.d_model = d_model
    self.n_layers = n_layers

    self.dec_layers = [DecoderLayer(d_model, n_heads, dff, rate) for _ in range(n_layers)]


  def call(self, x, enc_output, training, padding_mask, look_ahead_mask):
    attention_weights = {}

    for i in range(self.n_layers):
      x, block1, block2 = self.dec_layers[i](x, enc_output, training, padding_mask, look_ahead_mask)

      attention_weights['decoder_layer{}_block1'.format(i+1)] = block1
      attention_weights['decoder_layer{}_block2'.format(i+1)] = block2

    return x, attention_weights

In [ ]:
# 위치 임베딩 + 포지셔널 임베딩
embedded
#인코더 아웃풋 
Encoded

# 디코더의 위치+포지셔널 임베딩(Teaching Force 과정이라 디코더 부분에 타겟(영어)도 넣어줌)
target_embedder = TransformerEmbedding(d_model=512, input_vocab_size=tokenizer_en.vocab_size, max_pe_dim=512, dropout_rate=0.1)
target_embedding = target_embedder(tf.cast(np.random.uniform(size=(1,40)), dtype=tf.float32), True)

# 6층 디코더
Decoder_layer = Decoder(n_layers=6, d_model=512, n_heads=8, dff=2048)

Decoded, _ = Decoder_layer(target_embedding, Encoded, False, None, None) # Decoded에서 인코더와 디코더의 정보 결합
print(Decoded, Decoded.shape)
#최종출력
print("6층 Decoder 출력 :", sample_encoding.shape)

tf.Tensor(
[[[-0.3431721  -0.3290148   1.5598023  ...  2.303232    1.371545
    0.11695281]
  [-0.33767134 -0.33899462  1.5710102  ...  2.3005219   1.3617636
    0.11278047]
  [-0.3494037  -0.3352064   1.5691497  ...  2.297593    1.3669722
    0.11108976]
  ...
  [-0.35400254 -0.33930886  1.5467725  ...  2.3000638   1.3758032
    0.1186213 ]
  [-0.3487999  -0.33877504  1.543845   ...  2.3040543   1.3668836
    0.12298044]
  [-0.33305132 -0.34201398  1.5553851  ...  2.310325    1.3722295
    0.11848406]]], shape=(1, 40, 512), dtype=float32) (1, 40, 512)
6층 Decoder 출력 : (1, 40, 512)


# 임베딩 + 인코딩 + 디코딩을 결합하는 Transformer Class 정의  
최종적으로 지금까지 배웠던 임베딩, 인코딩, 디코딩을 결합하는 Transformer Class를 정의하겠습니다.

In [ ]:
class Transformer(tf.keras.Model):
    def __init__(self, n_layers, d_model, n_heads, dff, input_vocab_size, target_vocab_size, pe_input, pe_target, rate=0.1):
    
    # n_layers : 인코딩, 디코딩을 몇 층으로 할 지
    # d_model : 임베딩의 차원
    # n_heads : 병렬로 어텐션을 수행할 멀티 헤드 어텐션의 개수
    # dff : 포인트와이즈 피드포워드 네트워크에서 몇 차원의 연산이 이루어 질 지
    # input_vocab_size : 본 문제는 포르투갈어를 영어로 번역하는 문제이며, 포르투갈어 토크나이저의 총 단어수를 뜻함(처음 도입부에서 만들었음)
    # target_vocab_size : 영어 토크나이저의 총 단어수를 뜻함(위와 마찬가지)
    # pe_input : 별로 중요한 것은 아니지만, 위치 임베딩 할 때 위치 임베딩의 길이의 상한을 뜻함(포르투갈어)
    # pe_target : 위치 임베딩의 상한(영어)
        super().__init__()

        self.input_embedder = TransformerEmbedding(d_model, input_vocab_size, pe_input, rate) # 포르투갈어 임베딩
        self.encoder = Encoder(n_layers, d_model, n_heads, dff, rate) # x층 인코더

        self.target_embedder = TransformerEmbedding(d_model, target_vocab_size, pe_target, rate) # 영어 임베딩
        self.decoder = Decoder(n_layers, d_model, n_heads, dff, rate) # x층 디코더

        self.final_layer = Dense(target_vocab_size) # 최종 출력 decoder (영어 단어의 수), 우리가 맞추려는 것은 영어 단어의 인덱스이므로..

    def call(self, inp, tar, training):
        # inp : 포르투갈어
        # tar : 영어
        enc_mask = create_padding_mask(inp) # 인코더 패딩
        dec_mask = create_padding_mask(inp) # 디코더 패딩
        look_ahead_mask = create_masks(tar) # 디코더에 들어갈 look_ahead_mask 정의

        inp_embedding = self.input_embedder(inp) # 인코더 임베딩 정의
        enc_output = self.encoder(inp_embedding, training, enc_mask) # 인코더 아웃풋(디코더와 결합되게 됨)

        tar_embedding = self.target_embedder(tar) # 디코더 임베딩 정의

        dec_output, attention_weights = self.decoder(tar_embedding, enc_output, training, dec_mask, look_ahead_mask)

        final_output = self.final_layer(dec_output) # 최종 영어 단어를 예측하는 아웃풋 정의
        return final_output

In [ ]:
# 트랜스포머 테스트하기
sample_transformer = Transformer(6, 512, 8, 2048, tokenizer_pt.vocab_size + 2, tokenizer_en.vocab_size + 2, 10000, 10000)
inp = tf.cast(np.random.randint(100,size=40)[np.newaxis, :], dtype=tf.int32)
tar = tf.cast(np.random.randint(100,size=40)[np.newaxis, :], dtype=tf.int32)
transformer = sample_transformer(inp, tar, False)
print(transformer)

tf.Tensor(
[[[-0.3345034  -0.01396056  0.5463812  ... -0.05968201  0.3116074
   -0.4603326 ]
  [-0.33054727 -0.03038269  0.5541858  ... -0.04792291  0.3003717
   -0.46931484]
  [-0.3331985  -0.02435888  0.55642915 ... -0.04478735  0.3014708
   -0.4855488 ]
  ...
  [-0.3428004  -0.04054517  0.5622994  ... -0.05635282  0.2901105
   -0.49792284]
  [-0.34195268 -0.04063866  0.5711767  ... -0.06332943  0.2811991
   -0.49841833]
  [-0.34157473 -0.0376301   0.5684389  ... -0.06148854  0.28651989
   -0.50064284]]], shape=(1, 40, 8089), dtype=float32)


#하이퍼파라미터 설정하기
테스트를 위해서 층을 가볍게 쌓아보도록 하겠습니다.

In [160]:
input_vocab_size = tokenizer_pt.vocab_size + 2
target_vocab_size = tokenizer_en.vocab_size + 2
dropout_rate = 0.1

NameError: name 'tokenizer_pt' is not defined

# 옵티마이저 설정하기
옵티마이저는 논문에 따라서 성능이 좋았다는 옵티마이저를 복사 붙여넣기 하였습니다.

In [ ]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
  def __init__(self, d_model, warmup_steps=4000):
    super(CustomSchedule, self).__init__()
    
    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)

    self.warmup_steps = warmup_steps
    
  def __call__(self, step):
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps ** -1.5)
    
    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

learning_rate = CustomSchedule(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, 
                                     epsilon=1e-9)

# Loss 함수 설정하기
loss 함수 또한 중요한 부분인데, transformer에서는 패딩되는 부분을 Loss를 계산할 때 연산하지 않겠습니다.

In [ ]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none') #from_logits=True로 하면 Dense 이후 softmax layer 값 출력

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0)) # 예를 들어서 실제 자료(0은 패딩)가 [1,2,3,4,5,0,0,0,0,0] 이라면 [0,0,0,0,0,1,1,1,1,1]로 바꿔 줌
                                                     # 이후 tf.math.logical_not을 활용해서 [True,True,True,True,True,False,False,False,False,False]으로 바꿔 줌
  loss_ = loss_object(real, pred) # loss_는 패딩을 고려하지 않은 loss 값

  mask = tf.cast(mask, dtype=loss_.dtype) # [True,True,True,True,True,False,False,False,False,False]를 [1,1,1,1,1,0,0,0,0,0] 으로 바꿔 줌
  loss_ *= mask # loss에 mask를 곱해서, 패딩인 부분은 0처리 해줌

  return tf.reduce_sum(loss_)/tf.reduce_sum(mask)

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

# 모델 훈련

In [ ]:
transformer = Transformer(n_layers, d_model, n_heads, dff,
                          input_vocab_size, target_vocab_size, 
                          pe_input=10000, 
                          pe_target=10000,
                          rate=0.1)

In [ ]:
# 인풋, 아웃풋의 텐셔 shape 정의
train_step_signature = [
    tf.TensorSpec(shape=(None, None), dtype=tf.int64),
    tf.TensorSpec(shape=(None, None), dtype=tf.int64),
]

# tf.function을 사용하면 그래프를 미리 컴파일 하기 때문에 속도가 상당히 빠름
# 같은 GPU여도 케라스에 비해서 체감상 7~8배 정도의 차이가 나는 것 같음
@tf.function(input_signature=train_step_signature)
def train_step(inp, tar):
  tar_inp = tar[:, :-1]
  tar_real = tar[:, 1:]
  
  with tf.GradientTape() as tape:
    predictions = transformer(inp, tar_inp, True)
    loss = loss_function(tar_real, predictions)

  gradients = tape.gradient(loss, transformer.trainable_variables)    
  optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))
  
  train_loss(loss)
  train_accuracy(tar_real, predictions)

In [ ]:
# 저장할 체크포인트 지정
checkpoint_path = "./"

ckpt = tf.train.Checkpoint(transformer=transformer,
                           optimizer=optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

# if a checkpoint exists, restore the latest checkpoint.
if ckpt_manager.latest_checkpoint:
  ckpt.restore(ckpt_manager.latest_checkpoint)
  print ('Latest checkpoint restored!!')

In [ ]:
import time
# 20 에포크 훈련
for epoch in range(20):
  start = time.time()
  
  train_loss.reset_states()
  train_accuracy.reset_states()
  
  # input : 포루투갈어, tar : 영어
  for (batch, (inp, tar)) in enumerate(train_dataset):
    train_step(inp, tar)
    
    if batch % 50 == 0:
      print ('Epoch {} Batch {} Loss {:.4f} Accuracy {:.4f}'.format(
          epoch + 1, batch, train_loss.result(), train_accuracy.result()))
      
  if (epoch + 1) % 5 == 0:
    ckpt_save_path = ckpt_manager.save()
    print ('Saving checkpoint for epoch {} at {}'.format(epoch+1,
                                                         ckpt_save_path))
    
  print ('Epoch {} Loss {:.4f} Accuracy {:.4f}'.format(epoch + 1, 
                                                train_loss.result(), 
                                                train_accuracy.result()))

  print ('Time taken for 1 epoch: {} secs\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 8.9987 Accuracy 0.0000
Epoch 1 Batch 50 Loss 8.9453 Accuracy 0.0034
Epoch 1 Batch 100 Loss 8.8529 Accuracy 0.0134
Epoch 1 Batch 150 Loss 8.7466 Accuracy 0.0168
Epoch 1 Batch 200 Loss 8.6171 Accuracy 0.0184
Epoch 1 Batch 250 Loss 8.4618 Accuracy 0.0195
Epoch 1 Batch 300 Loss 8.2848 Accuracy 0.0236
Epoch 1 Batch 350 Loss 8.1004 Accuracy 0.0273
Epoch 1 Batch 400 Loss 7.9196 Accuracy 0.0301
Epoch 1 Batch 450 Loss 7.7574 Accuracy 0.0326
Epoch 1 Batch 500 Loss 7.6136 Accuracy 0.0355
Epoch 1 Batch 550 Loss 7.4792 Accuracy 0.0388
Epoch 1 Batch 600 Loss 7.3546 Accuracy 0.0427
Epoch 1 Batch 650 Loss 7.2348 Accuracy 0.0465
Epoch 1 Batch 700 Loss 7.1196 Accuracy 0.0503
Epoch 1 Loss 7.1152 Accuracy 0.0504
Time taken for 1 epoch: 79.08091878890991 secs

Epoch 2 Batch 0 Loss 5.7278 Accuracy 0.0938
Epoch 2 Batch 50 Loss 5.4732 Accuracy 0.1046
Epoch 2 Batch 100 Loss 5.4041 Accuracy 0.1061
Epoch 2 Batch 150 Loss 5.3532 Accuracy 0.1085
Epoch 2 Batch 200 Loss 5.3088 Accuracy 0.1099
Ep

# Inference
![Imgur](https://i.imgur.com/cUjg18g.png)  


평가는 훈련과는 다르게 진행됩니다.  
번역할 포르투갈어는 인코더 레이어를 거쳐 인코딩이 되고,  
디코더에는 영어 문장을 넣지 않고, 영어 문장의 시작 토큰만 디코더의 인풋으로 들어가게 됩니다.  
그러면 인코딩 된 것과 + 시작 토큰을 활용해서 다음 단어를 예측하고,  
인코딩 된 것 + 시작 토큰 + 전에 예측된 단어를 활용해서 다음 단어를 예측하는 방식입니다. 
  
그림에서는 bos가 시작 토큰입니다.
![Imgur](https://i.imgur.com/F6QseH6.png)

In [ ]:
def evaluate(inp_sentence):
  # inp_sentence : 문자 (string)
  start_token = [tokenizer_pt.vocab_size] # 포르투갈어의 시작 토큰
  end_token = [tokenizer_pt.vocab_size + 1] # 포르투갈어의 끝 토큰
  
  # 시작 토큰 + 포르투갈 어 + 끝 토큰
  inp_sentence = start_token + tokenizer_pt.encode(inp_sentence) + end_token
  encoder_input = tf.expand_dims(inp_sentence, 0)
  
  # 디코더의 인풋은 영어 문장의 시작 토큰만 들어감
  decoder_input = [tokenizer_en.vocab_size]
  output = tf.expand_dims(decoder_input, 0)
  
  for i in range(MAX_LENGTH):
    # predictions.shape == (batch_size, seq_len, vocab_size)
    predictions = transformer(encoder_input, output, False)
    
    # 예측 결과에서 마지막 부분만 추출
    predictions = predictions[: ,-1:, :]  # (batch_size, 1, vocab_size)

    predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

    # 만약에 예측된 영어 단어가 영어의 끝 토큰에 해당한다면 예측을 끝냄
    if predicted_id == tokenizer_en.vocab_size+1:
      return tf.squeeze(output, axis=0)
    
    # 예측된 단어를 전 단어와 결합하여 다음 예측에 써먹음
    output = tf.concat([output, predicted_id], axis=-1)

  return tf.squeeze(output, axis=0)

In [ ]:
def translate(sentence):
  result= evaluate(sentence)
  
  predicted_sentence = tokenizer_en.decode([i for i in result 
                                            if i < tokenizer_en.vocab_size])  

  print('Input: {}'.format(sentence))
  print('Predicted translation: {}'.format(predicted_sentence))

실제로 번역해보기  
제법 포르투갈 어를 영어 문법에 맞게 번역하는 것을 알 수 있습니다.

In [ ]:
translate("este é um problema que temos que resolver.")
print ("Real translation: this is a problem we have to solve .")

Input: este é um problema que temos que resolver.
Predicted translation: this is a problem we have to solve the united states .
Real translation: this is a problem we have to solve .


In [ ]:
translate("os meus vizinhos ouviram sobre esta ideia.")
print ("Real translation: and my neighboring homes heard about this idea .")

Input: os meus vizinhos ouviram sobre esta ideia.
Predicted translation: my neighbors heard about this idea .
Real translation: and my neighboring homes heard about this idea .


In [ ]:
translate("vou então muito rapidamente partilhar convosco algumas histórias de algumas coisas mágicas que aconteceram.")
print ("Real translation: so i 'll just share with you some stories very quickly of some magical things that have happened .")

Input: vou então muito rapidamente partilhar convosco algumas histórias de algumas coisas mágicas que aconteceram.
Predicted translation: so i 'm going to quickly share with you some stories of a few magic things that happened .
Real translation: so i 'll just share with you some stories very quickly of some magical things that have happened .


In [ ]:
translate("este é o primeiro livro que eu fiz.")
print ("Real translation: this is the first book i've ever done.")

Input: este é o primeiro livro que eu fiz.
Predicted translation: this is the first book that i did n't .
Real translation: this is the first book i've ever done.


출처  
http://jalammar.github.io/illustrated-gpt2/  
https://d2l.ai/chapter_recurrent-modern/seq2seq.html  
https://www.tensorflow.org/tutorials/text/transformer
